# How the intersubject activity evolves across levels

--- 

Q: How is the intersubject similarity is evolving across levels? If we pick a certain region, for a given subject, is the intersubject similarity higher or lower across levels? 

*One prediction*: if the region is involved in theory updating and learning then we expect a higher similarity for higher levels. As they learn more about the games (i.e. practice) subjects should converge to more similar voxel activation patterns. 

--- 

<img src="http://drive.google.com/uc?export=view&id=1FoQIRbghKP6RaNF8MKf_hM6wCmxABnXt" style="height:230px"/>



Task 2.1: Separate the betas as specified in notes and do a ISC between the respective vectors. Plot 9 brain maps & take the highest voxels.

Task 2.2: Get ROIs from my own data + momchils results and do the same analysis for these ROIs.
I want to make a plot like the one below (note: this is averaged across games and across subjects). One hypothesis is that over time (over levels) peoples representations become more similar (displayed by a larger ISC). In Bayesian terms, people start off with different priors but their posteriors will converge because they get more data. 
Caveat: this may not be the case, it may in fact be the opposite. 
APPROACH: pick the top ROI’s == the peak voxels. 
Take the voxels indicating ROI’s from momchils results (picture below) to test - Caveat: these are MNI coordinates! (so have to transform between voxel space and MNI space) [40.00]
1) for which of those voxels is the ISC significant across subjects
Smooth betas makes the voxels in the same neighbourhood more similar. Searchlight analysis: r = 2.6 voxels (4 mm); 4 voxels (6 mm); 6.6 voxels (10 mm)

3. Do a searchlight for the non smooth betas data

 Check methods in (Chen, 2016) and use this approach to do a SA.
Note: scenes in their paper corresponds to our blocks, levels, or games

In [1]:
import h5py
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import os 
import glob
import time
from copy import deepcopy
import numpy as np
import pandas as pd 

from nilearn import datasets, image
from nilearn import surface
from nilearn import plotting
from nilearn.input_data import NiftiMasker, NiftiLabelsMasker
import nibabel as nib

from brainiak import image, io
from brainiak.isc import isc, isfc, permutation_isc
from brainiak.isc import compute_summary_statistic
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d 
import seaborn as sns 
import pandas as pd
from importlib import reload 
import scipy.io as sio
from scipy import stats

# import own functions
import utils
reload(utils)

#%autosave 30
%matplotlib inline
sns.set(style = 'white', context='talk', font_scale=1, rc={"lines.linewidth": 2})

In [2]:
# specify local path
path = '/Users/Daphne/data/'

# load relevant data
levels_betas = np.load(path+'bold_data_levels.npy')
df_levels_betas = pd.read_csv(path+'ordered_betas_df_levels.csv') 

In [3]:
levels_betas.shape

(54, 220075, 8)

In [4]:
df_levels_betas.head(18)

,game,session,instance,level,0,1,2,3,4,5,...,220065,220066,220067,220068,220069,220070,220071,220072,220073,220074
0,bait_run_2_block_2,Sn(2),1,1,-0.636170,-0.373854,-1.497341,-2.112460,-2.415012,-2.243853,...,-0.038629,-0.039316,-0.151938,-0.356248,0.780166,0.548450,-0.330269,-0.504245,-0.659055,0.951636
1,bait_run_2_block_2,Sn(2),2,2,0.942014,1.007454,0.715748,0.992761,1.298554,1.549959,...,-0.713421,-0.823860,-1.147676,-1.684841,-1.002751,-1.144127,-0.899558,-1.241139,-1.615280,-0.130938
2,bait_run_2_block_2,Sn(2),3,3,1.655500,1.412734,1.227655,1.776052,2.057731,2.052981,...,0.045613,0.033627,-0.303758,-0.969602,2.104960,1.762077,-0.022836,-0.258680,-0.718477,2.578229
3,bait_run_4_block_1,Sn(4),1,4,-2.147773,-2.092517,-2.035357,-2.470369,-2.851994,-3.012785,...,-0.865872,-0.742881,-0.551392,-0.220907,2.100336,0.926276,-0.463729,-0.270517,-0.053412,0.738639
4,bait_run_4_block_1,Sn(4),2,5,-0.270154,-0.262434,-1.101391,-1.208300,-1.261242,-1.228580,...,-1.226742,-1.247940,-1.448989,-1.703236,0.070426,-0.780465,-0.973810,-0.965997,-1.032061,-0.639038
5,bait_run_4_block_1,Sn(4),3,6,0.459462,0.474201,0.329827,0.373864,0.371919,0.445141,...,-0.553346,-0.429069,-0.532006,-0.864090,0.715096,-0.121409,-0.475154,-0.437522,-0.479703,0.283229
6,bait_run_6_block_1,Sn(6),1,7,0.957387,0.738800,2.399410,2.961179,3.211064,3.122286,...,-0.760641,-0.884861,-1.113301,-1.390764,1.085315,0.449789,-0.591991,-0.801478,-1.021741,0.871723
7,bait_run_6_block_1,Sn(6),2,8,2.382568,1.996827,4.545879,5.526391,5.973599,5.794201,...,-0.860787,-0.881726,-0.995501,-1.354120,0.865629,0.570306,-0.424392,-0.579131,-0.756057,0.949078
8,bait_run_6_block_1,Sn(6),3,9,1.276682,0.799029,2.344363,3.259416,3.691774,3.456368,...,-2.243427,-2.508078,-2.809407,-3.172503,-1.113008,-1.643137,-1.424382,-1.855104,-2.319343,-0.706388
9,chase_run_1_block_2,Sn(1),1,1,2.191652,1.856000,2.749278,2.857435,2.615516,2.246860,...,0.636880,1.031873,1.476854,1.983294,3.921660,2.658031,0.345488,0.719358,1.134821,2.226359


In [16]:
# for each [54, voxels] betas data
# take the 1 st item and put together 
# take the 2nd item and append to something
# .... to the 9th item and append

# want: [54, voxels] --> [6 (games), 9 (levels)]


In [17]:
# take one subject 
s = 0

levels_betas_sub = levels_betas[:,:,s]

In [18]:
levels_betas.shape

(54, 220075, 8)

In [19]:
levels_betas_sub.shape # take one subjet betas arr

(54, 220075)

In [20]:
game_names = df_levels_betas['game'].values
game_levels = df_levels_betas['level'].values

print(game_names[0:9])
print(game_levels[0:9])
print(len(game_levels))

['bait_run_2_block_2' 'bait_run_2_block_2' 'bait_run_2_block_2'
 'bait_run_4_block_1' 'bait_run_4_block_1' 'bait_run_4_block_1'
 'bait_run_6_block_1' 'bait_run_6_block_1' 'bait_run_6_block_1']
[1 2 3 4 5 6 7 8 9]
54


In [32]:
# get every 1st item (all levels 1)
game_levels[8::9]

array([9, 9, 9, 9, 9, 9])

In [28]:
levels_betas_sub[0::9].shape

(6, 220075)

In [38]:
games_level_one = []
games_level_two = []
games_level_three = []
games_level_four = []
games_level_five = []
games_level_six = []
games_level_seven = []
games_level_eight = []
games_level_nine = []

# append every 
num_subjects = 8

for s in range(num_subjects):
    print(s)
    # take the array for that subject 
    levels_betas_sub = levels_betas[:,:,s]
    
    # level 1
    lvl_one_betas_sub = levels_betas_sub[0::9]
    games_level_one.append(lvl_one_betas_sub)
    
    # level 2
    lvl_two_betas_sub = levels_betas_sub[1::9]
    games_level_two.append(lvl_two_betas_sub)
    
    # level 3
    lvl_three_betas_sub = levels_betas_sub[2::9]
    games_level_three.append(lvl_three_betas_sub)
    
    # level 4
    lvl_four_betas_sub = levels_betas_sub[3::9]
    games_level_four.append(lvl_four_betas_sub)
    
    # level 5
    lvl_five_betas_sub = levels_betas_sub[4::9]
    games_level_five.append(lvl_five_betas_sub)
    
    # level 6
    lvl_six_betas_sub = levels_betas_sub[5::9]
    games_level_six.append(lvl_six_betas_sub)
    
    # level 7
    lvl_seven_betas_sub = levels_betas_sub[6::9]
    games_level_seven.append(lvl_seven_betas_sub)
    
    # level 8
    lvl_eight_betas_sub = levels_betas_sub[7::9]
    games_level_eight.append(lvl_eight_betas_sub)
    
    # level 9
    lvl_nine_betas_sub = levels_betas_sub[8::9]
    games_level_nine.append(lvl_nine_betas_sub)

0
1
2
3
4
5
6
7


In [48]:
betas_level_one = np.array(games_level_one)
betas_level_two = np.array(games_level_two)
betas_level_three = np.array(games_level_three)
betas_level_four = np.array(games_level_four)
betas_level_five = np.array(games_level_five)
betas_level_six = np.array(games_level_six)
betas_level_seven = np.array(games_level_seven)
betas_level_eight = np.array(games_level_eight)
betas_level_nine = np.array(games_level_nine)

In [49]:
betas_level_one.shape # [subjects, games, voxels]

(8, 6, 220075)

In [ ]:
# do isc for each level
# compute the isc correlations using the leave one out approach
isc_maps = isc(games_level_one, pairwise=False)